In [ ]:
pip install transformers


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
pip install sentencepiece


   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   --------------------- ------------------ 524.3/991.5 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 991.5/991.5 kB 4.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
pip install sacremoses


In [ ]:
import pandas as pd
import random
from collections import Counter
import re

# Step 1: Read the dataset from CSV
csv_file = "intent_category.csv"  # Replace with your actual file path
df = pd.read_csv(csv_file)

# Extract columns
instructions = df['instruction'].tolist()
intents = df['intent'].tolist()
categories = df['category'].tolist()

# Step 2: Define placeholders
placeholders = [
    "{{account type}}",
    "{{account category}}",
    "{{refund amount}}",
    "{{currency symbol}}",
    "{{invoice number}}",
    "{{person name}}"
]

# Step 3: Target distribution
target_count = 1500  # Each placeholder should appear exactly 1,500 times
placeholder_counts = {placeholder: 0 for placeholder in placeholders}

# Step 4: Inject placeholders into sentences (in-place)
for idx, sentence in enumerate(instructions):
    # Skip rows that already contain placeholders
    if any(placeholder in sentence for placeholder in placeholders):
        continue
    
    # Randomly select a placeholder that hasn't reached the target count
    available_placeholders = [ph for ph in placeholders if placeholder_counts[ph] < target_count]
    if not available_placeholders:
        break  # Stop if all placeholders have reached the target count
    
    placeholder = random.choice(available_placeholders)
    
    # Randomly decide where to inject the placeholder
    position = random.choice(["beginning", "middle", "end"])
    words = sentence.split()
    
    if position == "beginning":
        augmented_sentence = f"{placeholder} {sentence}"
    elif position == "middle":
        mid = len(words) // 2
        augmented_sentence = " ".join(words[:mid] + [placeholder] + words[mid:])
    else:  # position == "end"
        augmented_sentence = f"{sentence} {placeholder}"
    
    # Overwrite the original instruction with the augmented version
    instructions[idx] = augmented_sentence
    
    # Update the count for the selected placeholder
    placeholder_counts[placeholder] += 1

# Step 5: Update the DataFrame with augmented instructions
df['instruction'] = instructions

# Step 6: Save the augmented dataset to a new CSV file
output_csv_file = "augmented_dataset.csv"  # Replace with your desired output file path
df.to_csv(output_csv_file, index=False)

# Step 7: Validate the augmented dataset
def extract_placeholders(text):
    return re.findall(r'\{\{.*?\}\}', text)  # Match only valid placeholders

# Detect rows with placeholders
df['has_placeholder'] = df['instruction'].apply(lambda x: bool(extract_placeholders(x)))
rows_with_placeholders = df['has_placeholder'].sum()

# Count occurrences of each placeholder
placeholders_extracted = df[df['has_placeholder']]['instruction'].apply(extract_placeholders).explode()
placeholder_counts_final = Counter(placeholders_extracted)

# Convert to DataFrame
placeholder_df = pd.DataFrame.from_dict(placeholder_counts_final, orient='index', columns=['count'])

# Display final results
print(f"Rows with placeholders: {rows_with_placeholders}")
print("\nFinal Distribution of Placeholders:")
print(placeholder_df.sort_values(by='count', ascending=False))
print("\nTotal rows in the dataset:", len(df))


Rows with placeholders: 10581

Final Distribution of Placeholders:
                      count
{{person name}}        2387
{{account type}}       1786
{{account category}}   1704
{{refund amount}}      1696
{{currency symbol}}    1643
{{invoice number}}     1508

Total rows in the dataset: 13947
